<a href="https://colab.research.google.com/github/Mart1n610/CardioAlert/blob/main/CardioAlert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import google.colab as drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
señalPrueba = [1, 2, 3, 4, 5, 4, 3, 2, 1]
ventanaHamming = np.hamming(len(señal))
señalVentana = señalPrueba * ventanaHamming

In [ ]:
fourierft = np.fft.fft(señalVentana)
frecuenciaseñalp = 100 #frecuencia de prueba de 100 Hz
frecuenciaCardio = np.argmax(np.abs(fourierft)) * (frecuenciaseñalp / len(señalPrueba))